In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import torch
torch.cuda.empty_cache()
torch.cuda.device_count()

3

In [3]:
from sklearn.model_selection import KFold
from datasets import load_dataset, DatasetDict, Dataset, concatenate_datasets
import datasets
import pandas as pd
import os
import logging
import nltk
import numpy as np
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
import evaluate
from transformers import BartForConditionalGeneration, BartTokenizer
from random import sample


train_df = datasets.load_from_disk("/home/y.khan/cai6307-y.khan/Query-Focused-Tabular-Summarization/data/data/train")
test_df = datasets.load_from_disk("/home/y.khan/cai6307-y.khan/Query-Focused-Tabular-Summarization/data/data/test")
validate_df = datasets.load_from_disk("/home/y.khan/cai6307-y.khan/Query-Focused-Tabular-Summarization/data/data/validate")

In [4]:
model_path = "facebook/bart-large"
model = BartForConditionalGeneration.from_pretrained(model_path)
tokenizer = BartTokenizer.from_pretrained(model_path)

In [5]:
from typing import List, Dict

def tokenization_with_answer(examples):
    inputs = []
    
    task_prefix = "Given a query and a table, generate a summary that answers the query based on the information in the table: "

    for i, (query, table, summary) in enumerate(zip(examples['query'], examples['table'], examples['summary'])):
        flattened_table = flatten_table(table, i)
        input_text = f"{task_prefix} Table {flattened_table}. Query: {query}"

        inputs.append(input_text)
        
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True,padding='max_length')
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=512, truncation=True)
    
    model_inputs["labels"] = labels["input_ids"] 

    return model_inputs


def flatten_table(table: Dict, row_index: int) -> str:
    header = table.get('header', [])
    rows = table.get('rows', [])
    title = table.get('title', [])

    flattened_rows = []
    for i, row in enumerate(rows):
        row_text = f"Row {i}, " + ",".join([f"{col}:{val}" for col, val in zip(header, row)])
        flattened_rows.append("## "+row_text)

    flattened_table = f"Title: {' '.join(map(str, title))}" + " " + " ".join(flattened_rows)
    return flattened_table

tokenized_dataset_train = train_df.map(tokenization_with_answer, batched=True)
tokenized_dataset_test = test_df.map(tokenization_with_answer, batched=True)

processed_data_train = tokenized_dataset_train.remove_columns(['table','summary', 'row_ids', 'example_id', 'query'])
processed_data_test = tokenized_dataset_test.remove_columns(['table','summary', 'row_ids', 'example_id', 'query'])

Map: 100%|██████████| 500/500 [00:01<00:00, 307.27 examples/s]


In [6]:
def k_fold_split(dataset, num_folds=5):
    fold_size = len(dataset) // num_folds
    folds = []
    for i in range(num_folds):
        start = i * fold_size
        end = start + fold_size if i < num_folds - 1 else len(dataset)
        folds.append(dataset.select(range(start, end)))
    return folds

In [7]:
def postprocess_text(preds, labels):
        preds = [pred.strip() for pred in preds]
        labels = [label.strip() for label in labels]

        # rougeLSum expects newline after each sentence
        preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
        labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

        return preds, labels

def metric_fn(eval_predictions):
    predictions, labels = eval_predictions
    decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    for label in labels:
        label[label < 0] = tokenizer.pad_token_id  # Replace masked label tokens
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    decoded_predictions, decoded_labels = postprocess_text(decoded_predictions, decoded_labels)

    rouge = evaluate.load('rouge')

    # Compute ROUGE scores
    rouge_results = rouge.compute(predictions=decoded_predictions, references=decoded_labels)

    return rouge_results

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model= model)

train_args = Seq2SeqTrainingArguments(
    output_dir="./train_weights_bart",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    num_train_epochs=20,
    evaluation_strategy="epoch",
    save_strategy = "epoch",
    weight_decay=0.01,
    save_total_limit=5,
    warmup_ratio=0.03,
    load_best_model_at_end=True,
    predict_with_generate=True,
    overwrite_output_dir= True,
    gradient_accumulation_steps = 2
)

trainer = Seq2SeqTrainer(
    model,
    train_args,
    train_dataset=processed_data_train,
    eval_dataset=processed_data_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=metric_fn
)

In [8]:
folds = k_fold_split(train_df, num_folds=10)

for i in range(len(folds)):
    val_fold = folds[i]
    train_folds = [folds[j] for j in range(len(folds)) if j != i]
    train_dataset = concatenate_datasets(train_folds)

    tokenized_train = train_dataset.map(tokenization_with_answer, batched=True)
    tokenized_val = val_fold.map(tokenization_with_answer, batched=True)

    # Remove unnecessary columns
    processed_train = tokenized_train.remove_columns(['table', 'summary', 'row_ids', 'example_id', 'query'])
    processed_val = tokenized_val.remove_columns(['table', 'summary', 'row_ids', 'example_id', 'query'])

    # Update your trainer's train_dataset and eval_dataset
    trainer.train_dataset = processed_train
    trainer.eval_dataset = processed_val

    # Train your model
    trainer.train()
    trainer.evaluate()

Map: 100%|██████████| 200/200 [00:01<00:00, 117.90 examples/s]


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
0,No log,1.806160,0.275291,0.152713,0.235578,0.245544
2,No log,1.610214,0.284105,0.163069,0.239226,0.251037
4,No log,1.587574,0.295651,0.168796,0.250756,0.263469
6,No log,1.587405,0.279568,0.165480,0.243042,0.252933
8,No log,1.600482,0.292806,0.170324,0.253011,0.264448
10,No log,1.639196,0.290251,0.166524,0.247528,0.261092
12,No log,1.657281,0.289292,0.164774,0.247203,0.258371
14,1.313600,1.689944,0.288542,0.167620,0.248477,0.258131
16,1.313600,1.708887,0.286072,0.162800,0.245199,0.255929
18,1.313600,1.735466,0.289050,0.166605,0.248763,0.260246


Checkpoint destination directory ./train_weights_bart/checkpoint-37 already exists and is non-empty.Saving will proceed but saved results may be invalid.
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


Map: 100%|██████████| 200/200 [00:00<00:00, 383.43 examples/s]


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
0,No log,1.058101,0.312571,0.198014,0.271608,0.288192
2,No log,1.093894,0.312267,0.200840,0.275084,0.287292
4,No log,1.179435,0.302827,0.190454,0.266523,0.278482
6,No log,1.162406,0.306758,0.192912,0.270513,0.282561
8,No log,1.186453,0.311785,0.196796,0.271197,0.287473
10,No log,1.223825,0.312178,0.192999,0.269754,0.284220
12,No log,1.256767,0.304063,0.190302,0.267844,0.279882
14,0.890600,1.280182,0.315495,0.196581,0.273152,0.290909
16,0.890600,1.302007,0.315133,0.195838,0.274608,0.291525
18,0.890600,1.322521,0.311382,0.195466,0.272510,0.287894


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


Map: 100%|██████████| 200/200 [00:00<00:00, 369.38 examples/s]


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
0,No log,0.957841,0.315982,0.211709,0.281581,0.292776
2,No log,0.994784,0.311621,0.205208,0.272441,0.285070
4,No log,1.019915,0.309057,0.201398,0.272845,0.286624
6,No log,1.056413,0.313122,0.203740,0.279570,0.289700


Checkpoint destination directory ./train_weights_bart/checkpoint-37 already exists and is non-empty.Saving will proceed but saved results may be invalid.


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.21 GiB. GPU 0 has a total capacty of 79.15 GiB of which 412.19 MiB is free. Process 1144690 has 21.67 GiB memory in use. Process 2783718 has 19.02 GiB memory in use. Including non-PyTorch memory, this process has 38.04 GiB memory in use. Of the allocated memory 34.45 GiB is allocated by PyTorch, and 2.32 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
model.save_pretrained("BART")
tokenizer.save_pretrained("BART")

In [ ]:
valid1 = validate_df.select(range(0, 10))
valid2 = validate_df.select(range(10, 20))
valid3 = validate_df.select(range(20, 30))
valid4 = validate_df.select(range(30, 40))
valid5 = validate_df.select(range(40, 50))
valid6 = validate_df.select(range(50, 60))
valid7 = validate_df.select(range(60, 70))
valid8 = validate_df.select(range(70, 80))
valid9 = validate_df.select(range(80, 90))
valid10 = validate_df.select(range(90, 100))
valid11 = validate_df.select(range(100, 110))
valid12 = validate_df.select(range(110, 120))
valid13 = validate_df.select(range(120, 130))
valid14 = validate_df.select(range(130, 140))
valid15= validate_df.select(range(140, 150))
valid16 = validate_df.select(range(150, 160))
valid17 = validate_df.select(range(160, 170))
valid18 = validate_df.select(range(170, 180))
valid19 = validate_df.select(range(180, 190))
valid20 = validate_df.select(range(190, 200))
valid = [valid1, valid2, valid3, valid4, valid5, valid6, valid7, valid8, valid9, valid10, valid11, valid12, valid13, valid14, valid15, valid16, valid17, valid18, valid19,valid20]

In [ ]:
rougeL = []
for i in range(20):
    validate_df = valid[i].map(tokenization_with_answer, batched=True)
    predict_results = trainer.predict(validate_df, max_length = 1024)
    metrics = predict_results.metrics

    rougeL.append(metrics['test_rougeLsum'])
    
sum(rougeL)/20